In [ ]:
!pip install -U langchain-community
!pip install pypdf
!pip install chromadb
!pip install langchain
!pip install langchain-groq
!pip install sentence_transformers
!pip install numpy
!pip install gradio

In [ ]:
import os
import gradio as gr
from dotenv import load_dotenv
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.document_loaders import DirectoryLoader, PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_groq import ChatGroq
from langchain.chains import RetrievalQA, RefineDocumentsChain
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.chains.combine_documents.refine import RefineDocumentsChain
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.chains.combine_documents.map_reduce import MapReduceDocumentsChain
from langchain.chains.llm import LLMChain

In [7]:
def initialize_llm():
    return ChatGroq(
        temperature=0,
        groq_api_key=os.getenv("GROQ_API_KEY"),
        model_name="llama3-70b-8192"
    )


In [8]:
def create_vector_db():
    loader = DirectoryLoader("./data/", glob='*.pdf', loader_cls=PyPDFLoader)
    documents = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
    texts = text_splitter.split_documents(documents)
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    vector_db = Chroma.from_documents(texts, embeddings, persist_directory='./chroma_db')
    vector_db.persist()
    return vector_db

In [ ]:
load_dotenv()


# Başlat
llm = initialize_llm()
db_path = "./chroma_db/"
if not os.path.exists(db_path):
    vector_db = create_vector_db()
else:
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    vector_db = Chroma(persist_directory=db_path, embedding_function=embeddings)

retriever = vector_db.as_retriever()
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


C:\Users\yesil\AppData\Local\Temp\ipykernel_3064\692452992.py:10: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
C:\Users\yesil\AppData\Local\Temp\ipykernel_3064\692452992.py:11: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vector_db = Chroma(persist_directory=db_path, embedding

In [ ]:

# Prompt'lar
first_prompt = PromptTemplate(
    template="""
You are a compassionate and knowledgeable AI assistant trained to help individuals dealing with cyberbullying and mental health concerns.
Speak with the calm, empathetic, and reassuring tone of a professional psychologist.
Your responses should be warm, understanding, and supportive, helping the user feel safe and heard.
Provide practical advice, official steps, and online safety tips when relevant.
Always remind the user they are not alone and encourage seeking professional help if needed.

Rules to follow strictly:

1. Use gentle, non-judgmental language that validates the user’s feelings.
2. Avoid repeating the same points; keep responses clear, concise, and varied.
3. Do not give direct medical diagnoses or prescribe treatments; instead, encourage consulting a qualified professional when necessary.
4. Maintain confidentiality and never ask for sensitive personal information.
5. Stay away from controversial topics like politics or religion.
6. Respond in a calm, soothing manner that helps reduce anxiety.
7. Provide emotional support first, then practical advice.
8. When sharing official steps or safety tips, present them in an easy-to-understand way.
9. Always reassure the user that they are not alone and help is available.

Follow these guidelines to create a supportive and safe environment for users.

Here is an example of how you should respond:

---
User: Someone is sharing my private photos online without my permission. What should I do?
Chatbot: I'm really sorry you're going through this. First, make sure to take screenshots as evidence. You should report the content to the platform (like Instagram, TikTok, etc.) using their reporting tools. If the content poses a threat or you're underage, also consider informing a trusted adult or contacting local authorities. You're not alone in this — you're doing the right thing by reaching out.
---

Now, based on the context and your training data, respond to the following query:

{context}
User: {question}
Chatbot:""",
    input_variables=["context", "question"]
)

normal_prompt = PromptTemplate(
    template="""
{context}
User: {question}
Chatbot:""",
    input_variables=["context", "question"]
)

# Bu flag ilk mesajda True olacak
first_message = {"used": False}


In [21]:
# Chatbot cevabı
def chatbot_response(user_input, history):
    if not user_input.strip():
        return "Lütfen geçerli bir mesaj girin."

    try:
        prompt = first_prompt if not first_message["used"] else normal_prompt
        first_message["used"] = True  # bir sonraki için artık normal prompt kullan

        qa_chain = RetrievalQA.from_chain_type(
            llm=llm,
            chain_type="stuff",  # 'stuff' yerine bu stuff tüm veriyi birleştirip verir, halüsinasyona açık olur.
            retriever=retriever,
            chain_type_kwargs={"prompt": prompt}
        )

        response = qa_chain.run(user_input)
        return response

    except Exception as e:
        return f"Hata oluştu: {str(e)}"


Chain type Refine chain


In [ ]:

# Refine prompt'ları
question_prompt = PromptTemplate(
    input_variables=["context", "question"],
    template="""
You are a compassionate and knowledgeable AI assistant trained to help individuals dealing with cyberbullying and mental health concerns.
Speak with the calm, empathetic, and reassuring tone of a professional psychologist.
Your responses should be warm, understanding, and supportive, helping the user feel safe and heard.
Provide practical advice, official steps, and online safety tips when relevant.
Always remind the user they are not alone and encourage seeking professional help if needed.

Rules to follow strictly:

1. Use gentle, non-judgmental language that validates the user’s feelings.
2. Avoid repeating the same points; keep responses clear, concise, and varied.
3. Do not give direct medical diagnoses or prescribe treatments; instead, encourage consulting a qualified professional when necessary.
4. Maintain confidentiality and never ask for sensitive personal information.
5. Stay away from controversial topics like politics or religion.
6. Respond in a calm, soothing manner that helps reduce anxiety.
7. Provide emotional support first, then practical advice.
8. When sharing official steps or safety tips, present them in an easy-to-understand way.
9. Always reassure the user that they are not alone and help is available.
10. Use the context provided to answer the question, but do NOT copy the context verbatim.
11. Generate original, helpful, and empathetic responses based on the user’s question and the context.

Follow these guidelines to create a supportive and safe environment for users.

Use the following context to answer the question:
{context}

Question: {question}
Answer:"""
)

refine_prompt = PromptTemplate(
    input_variables=["context", "question", "existing_answer"],
    template="""
We have an existing answer based on earlier context:
{existing_answer}

Now, with the new context below, improve or expand the answer if needed.
If the context isn't helpful, keep the existing answer.

New Context:
{context}

Question: {question}
Refined Answer:"""
)

In [27]:
def chatbot_response(user_input, history):
    if not user_input.strip():
        return "Lütfen geçerli bir mesaj girin."

    try:
        # refine chain_type doğru parametrelerle kuruldu
                
        initial_chain = LLMChain(llm=llm, prompt=question_prompt)
        refine_chain = LLMChain(llm=llm, prompt=refine_prompt)

        # Refine chain
        combine_docs_chain = RefineDocumentsChain(
            initial_llm_chain=initial_chain,
            refine_llm_chain=refine_chain,
            document_variable_name="context",
            initial_response_name="existing_answer" 
        )

        # Retrieval QA zinciri
        qa_chain = RetrievalQA(
            retriever=retriever,
            combine_documents_chain=combine_docs_chain,
            return_source_documents=False
        )

        response = qa_chain.run(user_input)
        return response

    except Exception as e:
        return f"Hata oluştu: {str(e)}"

Map reduce


In [ ]:


# Map prompt (her doküman için)
map_chain = LLMChain(llm=llm, prompt=question_prompt)

# Reduce prompt (tüm map çıktılarının birleşimi)
reduce_prompt = PromptTemplate(
    input_variables=["text"],
    template="""
Tüm önceki yanıtları göz önünde bulundurarak aşağıdaki bilgiyi tek ve tutarlı şekilde cevapla:

{text}

Cevap:"""
)

reduce_llm_chain = LLMChain(llm=llm, prompt=reduce_prompt)

# StuffDocumentsChain ile reduce_llm_chain sarmalanır
reduce_chain = StuffDocumentsChain(
    llm_chain=reduce_llm_chain,
    document_variable_name="text"
)

# MapReduce zinciri oluştur
combine_docs_chain = MapReduceDocumentsChain(
    llm_chain=map_chain,
    reduce_documents_chain=reduce_chain,
    document_variable_name="context"
)

# Retrieval QA zinciri oluştur
qa_chain = RetrievalQA(
    retriever=retriever,
    combine_documents_chain=combine_docs_chain,
    return_source_documents=False
)

def chatbot_response(user_input, history):
    if not user_input.strip():
        return "Lütfen geçerli bir mesaj girin."

    try:
        response = qa_chain.run(user_input)
        return response
    except Exception as e:
        return f"Hata oluştu: {str(e)}"


In [53]:

# Gradio CSS
css = """
.gradio-container {
    background-image: url('data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQABAAD...');
    background-size: cover;
    background-position: center;
    background-repeat: no-repeat;
    background-attachment: fixed;
}
"""

# Arayüz
with gr.Blocks(css=css) as app:
    gr.ChatInterface(
        fn=chatbot_response,
        title="Mental Health Chatbot"
    )

app.launch()

c:\Users\yesil\AppData\Local\Programs\Python\Python311\Lib\site-packages\gradio\chat_interface.py:339: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


* Running on local URL:  http://127.0.0.1:7869
* To create a public link, set `share=True` in `launch()`.
